In [1]:
import dataprep.connector as dbc
import urllib
table_name = "injury_information"
mshp_db_path = r"C:\Users\lgarzia\Documents\GitHub\explainability\local_data\data\mshp.sqlite" 
mshp_db_path = urllib.parse.quote(mshp_db_path)
mshp_db_conn = "sqlite://"+mshp_db_path

In [13]:
sql = '''
WITH n AS
(
	SELECT acc_rpt_num, CASE WHEN max(veh_num) > 1 THEN 'MULTI' ELSE 'SINGLE' END AS num_veh
	FROM pre_feature_eng_base
	group by acc_rpt_num
)
SELECT e.*, num_veh 
FROM pre_feature_eng_base e JOIN n on e.acc_rpt_num = n.acc_rpt_num
'''

In [14]:
df = dbc.read_sql(mshp_db_conn, sql)

In [15]:
# Target Variable
df['injury_type'].value_counts(normalize=True)

MINOR        0.319291
NO INJURY    0.283677
MODERATE     0.201979
SERIOUS      0.145095
FATAL        0.049959
Name: injury_type, dtype: float64

In [16]:
df['injury_type'] = df['injury_type'].map({'MINOR': 0, 'NO INJURY': 0, 'MODERATE':0, 'SERIOUS':1, 'FATAL':1})

In [17]:
df.drop(['acc_rpt_num', 'acc_uuid', 'name', 'veh_num'], axis=1, inplace=True)

In [18]:
df

,injury_type,age,city_state,safety_device,date,time,troop,gender,involvement,vehicle_direction,num_veh
0,0,38,"HOWARDVILLE, MO",NO,03/06/2022,9:24PM,E,MALE,DRIVER,SOUTHBOUND,SINGLE
1,0,48,"PECULIAR, MO",YES,03/06/2022,9:15PM,A,MALE,DRIVER,NORTHBOUND,MULTI
2,0,38,"LATOUR, MO",YES,03/06/2022,9:15PM,A,MALE,DRIVER,NORTHBOUND,MULTI
3,0,59,"NORBORNE, MO",YES,03/06/2022,8:18PM,A,MALE,DRIVER,EASTBOUND,SINGLE
4,1,19,"CONWAY, MO",NO,03/06/2022,8:15PM,I,MALE,DRIVER,SOUTHBOUND,SINGLE
...,...,...,...,...,...,...,...,...,...,...,...
12125,0,74,"SPRINGFIELD, MO",YES,03/05/2023,12:45PM,D,MALE,DRIVER,NORTHBOUND,MULTI
12126,0,86,"NIXA, MO",YES,03/05/2023,12:45PM,D,FEMALE,DRIVER,EASTBOUND,MULTI
12127,0,27,"FLORISSANT, MO",NO,03/05/2023,3:57AM,C,MALE,DRIVER,EASTBOUND,SINGLE
12128,1,19,"RICHMOND, MO",NO,03/05/2023,1:40AM,A,MALE,DRIVER,WESTBOUND,SINGLE


In [23]:
# instate version out of state - low information mostly MO and long tail clean up
df['city_state'].str.split(',').str[-1].value_counts()

 MO               9098
 IL                264
 MISSOURI          235
MO                 203
 KS                201
                  ... 
LONE JACK            1
SHELL KNOB. MO       1
GILLESPIE IL         1
BERKELEY MO          1
CLINTON/MO           1
Name: city_state, Length: 544, dtype: int64

In [24]:
df.drop(['city_state'], axis=1, inplace=True)

In [25]:
df.head()

,injury_type,age,safety_device,date,time,troop,gender,involvement,vehicle_direction,num_veh
0,0,38,NO,03/06/2022,9:24PM,E,MALE,DRIVER,SOUTHBOUND,SINGLE
1,0,48,YES,03/06/2022,9:15PM,A,MALE,DRIVER,NORTHBOUND,MULTI
2,0,38,YES,03/06/2022,9:15PM,A,MALE,DRIVER,NORTHBOUND,MULTI
3,0,59,YES,03/06/2022,8:18PM,A,MALE,DRIVER,EASTBOUND,SINGLE
4,1,19,NO,03/06/2022,8:15PM,I,MALE,DRIVER,SOUTHBOUND,SINGLE


In [47]:
import pandas as pd
df['month_name'] = pd.to_datetime(df.date).dt.month_name()

In [42]:
# https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/
df['seconds'] = pd.to_datetime(df.time).dt.hour*60*60 +  pd.to_datetime(df.time).dt.minute*60
seconds_in_day = 24*60*60
import numpy as np
df['sin_time'] = np.sin(2*np.pi*df.seconds/seconds_in_day)
df['cos_time'] = np.cos(2*np.pi*df.seconds/seconds_in_day)

In [48]:
df.drop(['date', 'time', 'seconds'], axis=1, inplace=True)

In [49]:
df

,injury_type,age,safety_device,troop,gender,involvement,vehicle_direction,num_veh,seconds,sin_time,cos_time,month_name
0,0,38,NO,E,MALE,DRIVER,SOUTHBOUND,SINGLE,77040,-0.629320,0.777146,March
1,0,48,YES,A,MALE,DRIVER,NORTHBOUND,MULTI,76500,-0.659346,0.751840,March
2,0,38,YES,A,MALE,DRIVER,NORTHBOUND,MULTI,76500,-0.659346,0.751840,March
3,0,59,YES,A,MALE,DRIVER,EASTBOUND,SINGLE,73080,-0.824126,0.566406,March
4,1,19,NO,I,MALE,DRIVER,SOUTHBOUND,SINGLE,72900,-0.831470,0.555570,March
...,...,...,...,...,...,...,...,...,...,...,...,...
12125,0,74,YES,D,MALE,DRIVER,NORTHBOUND,MULTI,45900,-0.195090,-0.980785,March
12126,0,86,YES,D,FEMALE,DRIVER,EASTBOUND,MULTI,45900,-0.195090,-0.980785,March
12127,0,27,NO,C,MALE,DRIVER,EASTBOUND,SINGLE,14220,0.859406,0.511293,March
12128,1,19,NO,A,MALE,DRIVER,WESTBOUND,SINGLE,6000,0.422618,0.906308,March


In [50]:
df.gender.value_counts()

MALE       7489
FEMALE     4550
UNKNOWN      61
Name: gender, dtype: int64

In [51]:
df.involvement.value_counts()

DRIVER        10171
OCCUPANT       1803
PEDESTRIAN      140
BICYCLIST        14
OTHER             2
Name: involvement, dtype: int64

In [54]:
df['is_driver'] = np.where(df.involvement == 'DRIVER', 1, 0)

In [55]:
df.drop(['involvement'], axis=1, inplace=True)

In [56]:
df.head(n=6)

,injury_type,age,safety_device,troop,gender,vehicle_direction,num_veh,seconds,sin_time,cos_time,month_name,is_driver
0,0,38,NO,E,MALE,SOUTHBOUND,SINGLE,77040,-0.629320,0.777146,March,1
1,0,48,YES,A,MALE,NORTHBOUND,MULTI,76500,-0.659346,0.751840,March,1
2,0,38,YES,A,MALE,NORTHBOUND,MULTI,76500,-0.659346,0.751840,March,1
3,0,59,YES,A,MALE,EASTBOUND,SINGLE,73080,-0.824126,0.566406,March,1
4,1,19,NO,I,MALE,SOUTHBOUND,SINGLE,72900,-0.831470,0.555570,March,1
5,1,37,YES,H,FEMALE,EASTBOUND,SINGLE,71220,-0.892979,0.450098,March,1


In [62]:
df['vehicle_direction'] = df.vehicle_direction.str[:1].apply(lambda x: x if x in set(['E', 'W', 'N', 'S']) else 'U')

In [63]:
df.head(n=5)

,injury_type,age,safety_device,troop,gender,vehicle_direction,num_veh,seconds,sin_time,cos_time,month_name,is_driver
0,0,38,NO,E,MALE,S,SINGLE,77040,-0.629320,0.777146,March,1
1,0,48,YES,A,MALE,N,MULTI,76500,-0.659346,0.751840,March,1
2,0,38,YES,A,MALE,N,MULTI,76500,-0.659346,0.751840,March,1
3,0,59,YES,A,MALE,E,SINGLE,73080,-0.824126,0.566406,March,1
4,1,19,NO,I,MALE,S,SINGLE,72900,-0.831470,0.555570,March,1


In [64]:
df.troop.value_counts()

C    3379
D    2222
A    1869
E    1197
F    1037
I     718
G     617
B     554
H     537
Name: troop, dtype: int64

In [65]:
df.gender.value_counts()

MALE       7489
FEMALE     4550
UNKNOWN      61
Name: gender, dtype: int64

In [67]:
df

,injury_type,age,safety_device,troop,gender,vehicle_direction,num_veh,sin_time,cos_time,month_name,is_driver
0,0,38,NO,E,MALE,S,SINGLE,-0.629320,0.777146,March,1
1,0,48,YES,A,MALE,N,MULTI,-0.659346,0.751840,March,1
2,0,38,YES,A,MALE,N,MULTI,-0.659346,0.751840,March,1
3,0,59,YES,A,MALE,E,SINGLE,-0.824126,0.566406,March,1
4,1,19,NO,I,MALE,S,SINGLE,-0.831470,0.555570,March,1
...,...,...,...,...,...,...,...,...,...,...,...
12125,0,74,YES,D,MALE,N,MULTI,-0.195090,-0.980785,March,1
12126,0,86,YES,D,FEMALE,E,MULTI,-0.195090,-0.980785,March,1
12127,0,27,NO,C,MALE,E,SINGLE,0.859406,0.511293,March,1
12128,1,19,NO,A,MALE,W,SINGLE,0.422618,0.906308,March,1


In [69]:
import sqlite3 
mshp_db_path = r"C:\Users\lgarzia\Documents\GitHub\explainability\local_data\data\mshp.sqlite" 
conn = sqlite3.connect(mshp_db_path)
df.to_sql('mshp_modeling_data', conn)

12130